In [11]:
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

news=[]
url='https://news.naver.com/'
response = urllib.request.urlopen(url)
soup=BeautifulSoup(response,'html.parser')
results=soup.select('.section_list_ranking li a')
print(results)
for result in results:
    print('제목:',result.attrs['title'])
    print('링크:',result.attrs['href'])
    title=result.attrs['title']
    href=result.attrs['href']
    url_content='https://news.naver.com'+result.attrs['href']
    response_content = urllib.request.urlopen(url_content)
    soup_content=BeautifulSoup(response_content,'html.parser')
    content=soup_content.select_one('#articleBodyContents')
    print(content.contents)

    #가공작업
    output=''
    for item in content.strings:
        stripped = str(item).strip()    #공백제거
        if stripped=='':
            continue
        else:
             output+=item
    output=output.replace('본문 내용TV플레이어','')
    output=output.replace('// flash 오류를 우회하기 위한 함수 추가','')
    output=output.replace('function _flash_removeCallback() {}','')
    output=output.strip()
    news.append({'title':title,'href':href,'contents':output})
    
#print(news)    
dataframe=pd.DataFrame(news)
dataframe.to_csv("news.csv")

[<a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=052&amp;aid=0001426682&amp;date=20200410&amp;type=1&amp;rankingSeq=1&amp;rankingSectionId=100" title='[정면승부] 배우 박정숙 "결혼 전부터 당선인이던 남편 이재영, 흔들림없이 한결같은 사람"'>[정면승부] 배우 박정숙 "결혼 전부터 당선인이던 남편 이재영, 흔들림없이 한결같은 사람"</a>, <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=081&amp;aid=0003081592&amp;date=20200410&amp;type=1&amp;rankingSeq=2&amp;rankingSectionId=100" title="“투표율 높일라고 일부러 나왔어요”…사전투표 첫날, 역대 최고 투표율 왜?">“투표율 높일라고 일부러 나왔어요”…사전투표 첫날, 역대 최고 투표율 왜?</a>, <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nhn?mid=etc&amp;sid1=111&amp;rankingType=popular_day&amp;oid=437&amp;aid=0000235697&amp;date=20200410&amp;type=2&amp;rankingSeq=3&amp;rankingSectionId=100" title="&quot;주말에 한 방 터진다&quot;?…여야, 'n번방 폭로설' 신경전">"주말에 한 방 터진다"?…여야, 'n번방 폭로설' 신경전</a>, <a class="nclicks(rig.rankpol)" href="/main/ranking/read.nh